In [11]:
%run ./Imports_and_functions.ipynb

2.17.0
8
3.4.1
1 Physical GPUs, 1 Logical GPUs


# Face Recognition

## Triplet Loss

### Data

In [ ]:
def pick_random_excluding(input_list, exclude_value):
    # Filter the list to exclude the specified value
    filtered_list = [item for item in input_list if str(item) != exclude_value]
    
    # Check if the filtered list is not empty
    if not filtered_list:
        raise ValueError("No valid numbers available in the list after exclusion.")
    
    # Pick a random element from the filtered list
    return random.choice(filtered_list)

In [ ]:
def process_file(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (224,224))
    img = img.astype(np.float32)
    # img = img / 255.0

    return img

In [ ]:
def build_dataset():
    dataset_path = "/mnt/c/users/charl/Desktop/Celebrity Faces Dataset"
    celeb_list = os.listdir(dataset_path)
    
    anch = []
    pos = []
    neg = []
    triplet = []
    
    for celeb in celeb_list:
        image_celeb_list = glob.glob(os.path.join(dataset_path, celeb,"*.jpg"))
    
        for image_celeb in image_celeb_list:
            anch.append(process_file(image_celeb))
            pos.append(process_file(pick_random_excluding(image_celeb_list,image_celeb)))
    
            celeb_neg = pick_random_excluding(celeb_list,celeb)
            image_celeb_neg_list = glob.glob(os.path.join(dataset_path, celeb_neg,"*.jpg"))
    
            neg.append(process_file(pick_random_excluding(image_celeb_neg_list,image_celeb)))

    anch, pos, neg = anch[0:128], pos[0:128], neg[0:128]

    for i in range(0, 128):
        triplet.append([anch[i], pos[i], neg[i]])
        
    return tf.convert_to_tensor(triplet)

In [ ]:
def build_generator():
    for triplet in triplets:
        yield (triplet[0], triplet[1], triplet[2]), 0.0    

In [ ]:
triplets = build_dataset()

In [ ]:
dataset = tf.data.Dataset.from_generator(
    build_generator,
    output_signature=(
        (
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)
        ),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)
dataset = dataset.batch(32)

### Model

In [ ]:
def triplet_loss(y_true, y_pred):
    # Unpack the embeddings
    anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)
    
    # Compute pairwise distances
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)  # Distance between anchor and positive
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)  # Distance between anchor and negative
    
    # Compute the triplet loss
    loss = tf.maximum(pos_dist - neg_dist + 0.2, 0.0)  # Margin of 0.2
    return tf.reduce_mean(loss)

In [ ]:
Res = resnet.ResNet50(include_top = False,
                      weights = None,
                      input_shape=(224,224,3))
Res.trainable = True

x = tf.keras.layers.GlobalAveragePooling2D()(Res.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
base_res = Model(inputs = Res.input, outputs = x, name = "embedding")

In [ ]:
anchor = tf.keras.Input(shape = (224,224,3), name = "anchor")
positive = tf.keras.Input(shape = (224,224,3), name = "positive")
negative = tf.keras.Input(shape = (224,224,3), name = "negative")

input = [anchor, positive, negative]

In [ ]:
anchor_emb = base_res(anchor)
positive_emb = base_res(positive)
negative_emb = base_res(negative)

output = tf.keras.layers.Concatenate()([anchor_emb, positive_emb, negative_emb])
# output = [anchor_emb, positive_emb, negative_emb]

In [ ]:
siamese_network = Model(inputs=input, outputs=output, name="SIAMESE")
siamese_network.summary()

In [ ]:
siamese_network.compile(optimizer = "adam",
                         loss = triplet_loss)

In [ ]:
tf.keras.config.disable_traceback_filtering()
siamese_network.fit(dataset, epochs=10, batch_size=32, verbose=1)

### Test

In [ ]:
anchor_path = "/mnt/c/users/charl/Desktop/Celebrity Faces Dataset/Denzel Washington/018_e4ed6557.jpg"
positive = "/mnt/c/users/charl/Desktop/test/denzel_washington_test.jpg"
negative = "/mnt/c/users/charl/Desktop/test/jamie_foxx_test.jpg"

test_a = process_file(anchor_path)
test_a = tf.convert_to_tensor(test_a)
test_a = tf.expand_dims(test_a, axis = 0)

test_p = process_file(positive)
test_p = tf.convert_to_tensor(test_p)
test_p = tf.expand_dims(test_p, axis = 0)

test_n = process_file(negative)
test_n = tf.convert_to_tensor(test_n)
test_n = tf.expand_dims(test_n, axis = 0)

In [ ]:
embedding = siamese_network.predict([test_a, test_p, test_n])

In [ ]:
embedding.shape

In [ ]:
pred_a, pred_p, pred_n = tf.split(embedding, num_or_size_splits=3, axis=1)

In [ ]:
anchor_pos = np.linalg.norm(pred_a - pred_p)
anchor_neg = np.linalg.norm(pred_a - pred_n)

In [ ]:
anchor_pos

In [ ]:
anchor_neg

## Contrastive loss

### Data

In [ ]:
count_images_data("CelebrityFaces")
count_images_data("FootballFaces")

In [2]:
image_shape = (75, 75, 3)
batch_size = 64
id = datetime.now()

# double_p_train = create_double_pos_v2(folder_name="CelebrityFaces",
#                                       set_="train",
#                                       save_data=True,
#                                       id=id,
#                                       count_per_image=50,
#                                       apply_augmentation=True)

# double_n_train = create_double_neg_v2(folder_name="CelebrityFaces",
#                                       set_="train",
#                                       save_data=True,
#                                       id=id,
#                                       count_per_image=50,
#                                       apply_augmentation=True)

double_p_train, double_n_train = load_dataset_from_dir("2024_10_20_54_21", "train")

buffer_size = len(double_n_train)

y_pos = tf.zeros((len(double_p_train),1))
y_neg =  tf.ones((len(double_n_train),1))

doubles_train = tf.concat([double_p_train, double_n_train], axis=0)
ys_train = tf.concat([y_pos, y_neg], axis = 0)

step = math.ceil(doubles_train.numpy().shape[0]/batch_size)

doubles_train, ys_train = memory_eff_shuffling(doubles_train, ys_train)

dataset_train = create_dataset_v2(doubles_train, ys_train, image_shape, "train",batch_size, buffer_size)

dataset shuffled


In [ ]:
doubles_train

In [ ]:
ShowRandomDoublet(dataset_train,1)

In [3]:
# double_p_valid = create_double_pos_v2(folder_name="FootballFaces",
#                                       set_="test",
#                                       save_data=True,
#                                       id=id,
#                                       count_per_image=2,
#                                       apply_augmentation=False)

# double_n_valid = create_double_neg_v2(folder_name="FootballFaces",
#                                       set_="test",
#                                       save_data=True,
#                                       id=id,
#                                       count_per_image=2,
#                                       apply_augmentation=False)

double_p_valid, double_n_valid = load_dataset_from_dir("2024_10_20_54_21", "test")

y_pos = tf.zeros((len(double_p_valid),1))
y_neg =  tf.ones((len(double_n_valid),1))

doubles_valid = tf.concat([double_p_valid, double_n_valid], axis=0)
ys_valid = tf.concat([y_pos, y_neg], axis = 0)

doubles_valid, ys_valid = memory_eff_shuffling(doubles_valid, ys_valid)

dataset_valid = create_dataset_v2(doubles_valid, ys_valid, image_shape, "valid",batch_size, None)

In [ ]:
ShowRandomDoublet(dataset_valid,1)

In [ ]:
doubles_valid

### Model

#### Simple Feature Extractor

In [ ]:
l2_reg = regularizers.l2(0.08)

input_emb = tf.keras.Input(shape=image_shape)
  
x = tf.keras.layers.Conv2D(64, (11,11), activation="relu", padding = "same", kernel_regularizer=l2_reg, kernel_initializer=HeNormal())(input_emb)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D((2,2))(x)

x = tf.keras.layers.Conv2D(128, (7,7), activation="relu", padding="same", kernel_regularizer=l2_reg, kernel_initializer=HeNormal())(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D((2,2))(x)

x = tf.keras.layers.Conv2D(256, (4,4), activation="relu", padding="same", kernel_regularizer=l2_reg, kernel_initializer=HeNormal())(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D((2,2))(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(832, activation = "relu", kernel_regularizer=l2_reg, kernel_initializer=HeNormal())(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Dense(576, activation = "relu", kernel_regularizer=l2_reg, kernel_initializer=HeNormal())(x)
x = tf.keras.layers.Dropout(0.2)(x)

output = tf.keras.layers.Dense(512, kernel_regularizer=l2_reg, kernel_initializer=GlorotUniform())(x)
output = tf.keras.layers.Lambda(lambda output: tf.math.l2_normalize(output, axis=-1))(output)

base_model = Model(inputs=input_emb, outputs=output, name="embedding")
base_model.summary()

In [ ]:
first_embedding_in = tf.keras.Input(shape = image_shape)
second_embedding_in = tf.keras.Input(shape = image_shape)
input_list = [first_embedding_in, second_embedding_in]

first_embedding = base_model(first_embedding_in)
second_embedding = base_model(second_embedding_in)

output = tf.keras.layers.Lambda(cosine_distance)([first_embedding, second_embedding])

siamese_network = Model(inputs = input_list, outputs=output, name = "siamese_network")
siamese_network.summary()

In [ ]:
dataset_path = "/mnt/c/users/charl/Desktop/Face_Recognition_Siamese/Data/clean/CelebrityFaces"
anchor, positif, negatif = generate_triplet_string(dataset_path)

generate_distance_embbedding(anchor=anchor,
                             positif=positif,
                             negatif=negatif,
                             embedding_model=base_model,
                             distance_calculation = siamese_network)

In [ ]:
#Method : mean - median - 3rd_quartile
initial_learning_rate = 1e-5
decay_rate = 0.92  # You can adjust this

# Learning rate schedule: decay every 2 epochs
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=step * 2,  # decay every 2 epochs
    decay_rate=decay_rate,
    staircase=True  # Ensures it decreases in a step-wise manner
)

optimizer = Adam(learning_rate=lr_schedule)

model, T = fit(epochs = 60,
               training_data=dataset_train ,
               validation_data=dataset_valid,
               model=siamese_network,
               loss_func=contrastive_loss,
               optimizer=optimizer,
               grouping_method="median",
               training_step=step)

#### MobileNetV3Large

In [ ]:
base_model = MobileNetV3Large(include_top = False,
                              weights = "imagenet",
                              input_shape=image_shape,
                              alpha = 1.0)

base_model.summary()

In [ ]:
layers = []
for layer in base_model.layers:
    name = layer.name
    layers.append(name)

pourcentages = [0.1, 0.25, 0.50, 0.70, 0.90]
freezing_checkpoints = []

for pourcent in pourcentages:
    ckpt_index = int(pourcent * len(layers))
    ckpt = layers[ckpt_index]
    freezing_checkpoints.append(ckpt)

freezing_checkpoints

In [ ]:
# Set layers to non-trainable up to 'block_6_project_BN'
layer_name = 'expanded_conv_1_project'
freeze = True
for layer in base_model.layers:
    if layer.name == layer_name:
        print("Freezing")
        freeze = False
    layer.trainable = not freeze

In [ ]:
siamese_network = instanciate_model(base_model=base_model,
                                    image_shape=image_shape,
                                    apply_reg=True,
                                    reg_weight=0.09,
                                    apply_dropout=True,
                                    dropout_prob=0.3,
                                    apply_augmentation=False,
                                    augmentation_model=None,
                                    distance_method="cosine",
                                    embedding = 512)
siamese_network.summary()

In [ ]:
#Method : mean - median - 3rd_quartile
initial_learning_rate = 1e-5
decay_rate = 0.96  # You can adjust this

# Learning rate schedule: decay every 2 epochs
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=step * 2,  # decay every 2 epochs
    decay_rate=decay_rate,
    staircase=True  # Ensures it decreases in a step-wise manner
)

optimizer = Adam(learning_rate=lr_schedule)

model, T = fit(epochs = 18,
               training_data=dataset_train ,
               validation_data=dataset_valid,
               model=siamese_network,
               loss_func=contrastive_loss,
               optimizer=optimizer,
               grouping_method="median",
               training_step=step)

In [ ]:
T

In [ ]:
acc = calculate_accuracy(siamese_network, dataset_valid,  0.47)
acc

#### MobileNetV3Small

In [4]:
base_model = MobileNetV3Small(include_top = False,
                              weights = "imagenet",
                              input_shape=image_shape,
                              alpha = 1.0)

base_model.summary()

Model: "MobileNetV3Small"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 75, 75, 3) │          0 │ input_layer[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 38, 38,    │        432 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 38, 38,    │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 38, 38,    │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 39, 39,    │          0 │ activation[0][0]  │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 19, 19,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 19, 19,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 19, 19,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 1, 1, 16)  │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 1, 1, 16)  │          0 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 19, 19,    │          0 │ re_lu[0][0],      │
│ (Multiply)          │ 16)               │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 19, 19,    │        256 │ expanded_conv_sq

 Total params: 939,120 (3.58 MB)

 Trainable params: 927,008 (3.54 MB)

 Non-trainable params: 12,112 (47.31 KB)

In [5]:
layers = []
for layer in base_model.layers:
    name = layer.name
    layers.append(name)

pourcentages = [0.1, 0.25, 0.50, 0.70, 0.90]
freezing_checkpoints = []

for pourcent in pourcentages:
    ckpt_index = int(pourcent * len(layers))
    ckpt = layers[ckpt_index]
    freezing_checkpoints.append(ckpt)

freezing_checkpoints

['expanded_conv_project',
 'expanded_conv_3_depthwise',
 'expanded_conv_5_project_bn',
 'expanded_conv_8_expand',
 'activation_15']

In [6]:
# Set layers to non-trainable up to 'block_6_project_BN'
layer_name = 'expanded_conv_3_depthwise'
freeze = True
for layer in base_model.layers:
    if layer.name == layer_name:
        print("Freezing")
        freeze = False
    layer.trainable = not freeze

Freezing


In [ ]:
aug = augmentation_model()

In [7]:
siamese_network = instanciate_model(base_model=base_model,
                                    image_shape=image_shape,
                                    apply_reg=True,
                                    reg_weight=0.09,
                                    apply_dropout=True,
                                    dropout_prob=0.4,
                                    apply_augmentation=False,
                                    augmentation_model=None,
                                    distance_method="cosine",
                                    embedding = 256)
siamese_network.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 75, 75, 3) │          0 │ input_layer[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 38, 38,    │        432 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 38, 38,    │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 38, 38,    │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 39, 39,    │          0 │ activation[0][0]  │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 19, 19,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 19, 19,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 19, 19,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 1, 1, 16)  │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 1, 1, 16)  │          0 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 19, 19,    │          0 │ re_lu[0][0],      │
│ (Multiply)          │ 16)               │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 19, 19,    │        256 │ expanded_conv_sq

 Total params: 1,907,312 (7.28 MB)

 Trainable params: 1,882,216 (7.18 MB)

 Non-trainable params: 25,096 (98.03 KB)

Model: "siamese_network"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 256)       │  1,907,312 │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ embedding[0][0],  │
│                     │                   │            │ embedding[1][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,907,312 (7.28 MB)

 Trainable params: 1,882,216 (7.18 MB)

 Non-trainable params: 25,096 (98.03 KB)

In [12]:
#Method : mean - median - 3rd_quartile
initial_learning_rate = 1e-5
decay_rate = 0.96  # You can adjust this

# Learning rate schedule: decay every 2 epochs
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=step * 2,  # decay every 2 epochs
    decay_rate=decay_rate,
    staircase=True  # Ensures it decreases in a step-wise manner
)

optimizer = Adam(learning_rate=lr_schedule)

model, T = fit(epochs = 50,
               training_data=dataset_train ,
               validation_data=dataset_valid,
               model=siamese_network,
               loss_func=contrastive_loss,
               optimizer=optimizer,
               grouping_method="median",
               training_step=step)

Training Starting...


2024-10-20 09:01:22.754623: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 9533 of 18265
2024-10-20 09:01:31.919745: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


Step 569/569  

2024-10-20 09:03:06.667647: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Threshold Determined by median: 0.6094673275947571 
EPOCH 1/50 - training accuracy: 0.66 training loss: 0.23 validation accuracy: 0.55 validation_loss: 0.34



...

Step 569/569  Threshold Determined by median: 0.29428809881210327 
EPOCH 48/50 - training accuracy: 0.72 training loss: 0.14 validation accuracy: 0.53 validation_loss: 0.35



2024-10-20 10:14:53.793824: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 9760 of 18265
2024-10-20 10:15:02.525052: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


Step 569/569  Threshold Determined by median: 0.29091426730155945 
EPOCH 49/50 - training accuracy: 0.72 training loss: 0.14 validation accuracy: 0.55 validation_loss: 0.37



2024-10-20 10:16:22.904398: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 9829 of 18265
2024-10-20 10:16:31.449337: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


Step 569/569  Threshold Determined by median: 0.29102402925491333 
EPOCH 50/50 - training accuracy: 0.73 training loss: 0.14 validation accuracy: 0.54 validation_loss: 0.36



In [ ]:
acc = calculate_accuracy(siamese_network, dataset_valid,  0.55)
acc

In [ ]:
dataset_path = "/mnt/c/users/charl/Desktop/Face_Recognition_Siamese/Data/clean/CelebrityFaces"
anchor, positif, negatif = generate_triplet_string(dataset_path)

generate_distance_embbedding(anchor=anchor,
                             positif=positif,
                             negatif=negatif,
                             embedding_model=base_model,
                             distance_calculation = siamese_network)

## Test